In [ ]:
"""
References to functions in the code:
integer_modulo(x,n) # x mod n
polynomial_modulo(p,n,v) # p mod n
polynomial_division(p,d,v) # p / d, return [quotient, remainder]
def GF(p,q,v)
degree_of_q_polynomials(p,q,v)
find_irreducible_polynomial(p,q,v) # irreducible polynomials of degree q

---- find compositional irreducible polynomials mod 2 (composed with all polynomials in the field)(exclude constants) ----
find_compositional_irreducible_mod2_by_allPoly(irre,p,q,v)

"""

In [2]:
def integer_modulo(x,n):
    if n<=0:
        return "n must be a positive integer"
    r=x
    while r>=n:
        r=r-n
    while r<0:
        r=r+n
    return r

In [3]:
def polynomial_modulo(p,n,v):
    if n<0:
        return "n must be a positive integer"
    newp=0
    for i in range (len(p.coefficients(v))):
        newp=newp+integer_modulo(p.coefficients(v)[i][0],n)*v^p.coefficients(v)[i][1]
    return newp

In [4]:
# p is the dividend; d is the divisor; v is the variable; output[q,r], quotient and remainder
def polynomial_division(p,d,v):
    if p.degree(v)>=1 and d.degree(v)>=1:
        p=p.expand()
        d=d.expand()
        q=0
        r=p
        while p.degree(v)>=d.degree(v):
            L=(p.coefficient(v^p.degree(v))/d.coefficient(v^d.degree(v)))*v^(p.degree(v)-d.degree(v))
            p=p-(L*d).expand()
            q=q+L
        r=p
        return [q,r]
    else:
        if d.degree(v)==0:
            if d==0*v^0:
                return "Invalid input, can't divided by zero!"
            else:
                q=p/d
                r=0
                return[q,r]
        if p.degree(v)==0:
            q=0;
            r=p;
            return[q,r]

In [5]:
def GF(p,q,v):
    poly=[];
    if q!=1:
        for i in range(p):
            new=[i*v^(q-1) + k for k in GF(p,q-1,v)]
            poly=poly+new
    else:
        for i in range(p):
            poly.append(i)
    return poly

In [6]:
def degree_of_q_polynomials(p,q,v):
    if q<1:
        return "q should >= 1"
    field=GF(p,q,v)
    poly=[]
    for i in range(1,p):
        for j in field:
            poly.append(i*v^q+j)
    return poly

In [7]:
#Outputs all the irreducible polynomials in the varible v that with coefficients less than p, and degree of q
def find_irreducible_polynomial(p,q,v):
    field = GF(p,q,v) 
    
    poly=[] # the same as poly = degree_of_q_polynomials(p,q,v), but more efficient
    for i in range(1,p):
        for j in field:
            poly.append(i*v^q+j)
    
    for i in range(p): # remove all constants, which cannot produce a polynomial of degree q
        field.remove(i)
    
    for i in field:
        for j in range( p ** (q - i.degree(v)) - p, len(field)): # polynomial with smaller degree cannot produce a polynomial of degree q
            result = polynomial_modulo((i * field[j]).expand(),p,v)
            if result.degree(v) == q and result in poly: # remove reducible polynomial and avoid multiple removing of the same polynomial
                poly.remove(result) 
    return poly

In [8]:
from itertools import combinations 
def find_compositional_irreducible_mod2_by_allPoly(irre,p,q,v):
    field = GF(p,q,v)
    
    field_of_degree_q = []
    for i in range(1,p):
        for j in field:
            field_of_degree_q.append(i*v^q+j)

    # remove all constants
    field.remove(0)
    field.remove(1)
    
    for i in field: # to compose with i
        
        individual_compo = [] # store all composition polynomials by compose i with one non-constant polynomial in the field
        for j in field:
            new=j.subs(v==i)
            new=polynomial_modulo(new,2,v)
            if new not in individual_compo:
                individual_compo.append(new)
        
        
        compo = [] # store all possible combinations of component composition polynomials
        for j in range(1,len(individual_compo)+1):
            new2=list(combinations(individual_compo, j))
            
            compo=compo+new2
           
        
        
        for j in range(len(compo)):
            new3=0
            print(len(compo[j]))
            for k in range(len(compo[j])):
                new3=new3+compo[j][k]
            
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
            new3 = polynomial_division(new3,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
            new3 = polynomial_modulo(new3,2,v)
            if new3.degree(v) == q and new3 in field_of_degree_q:
                field_of_degree_q.remove(new3)
    return field_of_degree_q
    

In [9]:
def find_five(irre,v):
    print(irre)
    field = GF(2,5,v);
    pool = GF(2,5,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^5+j)

    for mother in reversed(field):
        

        coe = mother.coefficients()
        if (len(coe)>=5):
            four = coe[4][0] 
        else:
            four = 0
        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        for a in range(32):
            for b in range(32):
                for c in range(32):
                    for d in range (32):
                        child = expand(four*(pool[a])^4+three*(pool[b])^3+two*(pool[c])^2+one*(pool[d])^1+zero)
                        if child.degree(v)<=4:
                            break
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                        child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                        child = polynomial_modulo(child,2,v)
                        if child.degree(v) == 5 and child in field_of_degree_q:
                            field_of_degree_q.remove(child)
                        if len(field_of_degree_q)==0:
                            return []
                        
    return field_of_degree_q
            

    

In [17]:
v = var('v')
for irre in degree_of_q_polynomials(2,6,v):
    poly = find_five(irre,v)
    print(poly)

v^6
[]
v^6 + 1
[]
v^6 + v
[]
v^6 + v + 1
[]
v^6 + v^2
[]
v^6 + v^2 + 1
[]
v^6 + v^2 + v
[]
v^6 + v^2 + v + 1
[]
v^6 + v^3
[]
v^6 + v^3 + 1
[]
v^6 + v^3 + v
[]
v^6 + v^3 + v + 1
[]
v^6 + v^3 + v^2
[]
v^6 + v^3 + v^2 + 1
[]
v^6 + v^3 + v^2 + v
[]
v^6 + v^3 + v^2 + v + 1
[]
v^6 + v^4
[]
v^6 + v^4 + 1
[]
v^6 + v^4 + v
[]
v^6 + v^4 + v + 1
[]
v^6 + v^4 + v^2
[]
v^6 + v^4 + v^2 + 1
[]
v^6 + v^4 + v^2 + v
[]
v^6 + v^4 + v^2 + v + 1
[]
v^6 + v^4 + v^3
[]
v^6 + v^4 + v^3 + 1
[]
v^6 + v^4 + v^3 + v
[]
v^6 + v^4 + v^3 + v + 1
[]
v^6 + v^4 + v^3 + v^2
[]
v^6 + v^4 + v^3 + v^2 + 1
[]
v^6 + v^4 + v^3 + v^2 + v
[]
v^6 + v^4 + v^3 + v^2 + v + 1
[]
v^6 + v^5
[]
v^6 + v^5 + 1
[]
v^6 + v^5 + v
[]
v^6 + v^5 + v + 1
[]
v^6 + v^5 + v^2
[]
v^6 + v^5 + v^2 + 1
[]
v^6 + v^5 + v^2 + v
[]
v^6 + v^5 + v^2 + v + 1
[]
v^6 + v^5 + v^3
[]
v^6 + v^5 + v^3 + 1
[]
v^6 + v^5 + v^3 + v
[]
v^6 + v^5 + v^3 + v + 1
[]
v^6 + v^5 + v^3 + v^2
[]
v^6 + v^5 + v^3 + v^2 + 1
[]
v^6 + v^5 + v^3 + v^2 + v
[]
v^6 + v^5 + v^3 + v^2 + v

In [18]:
def find_six(irre,v):
    print(irre)
    field = GF(2,6,v);
    pool = GF(2,6,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^6+j)

    for mother in reversed(field):
        

        coe = mother.coefficients()
        if (len(coe)>=6):
            five = coe[5][0] 
        else:
            five = 0
        if (len(coe)>=5):
            four = coe[4][0] 
        else:
            four = 0
        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        for a in range(64):
            for b in range(64):
                for c in range(64):
                    for d in range (64):
                        for e in range(64):
                            child = expand(five*(pool[a])^5+four*(pool[b])^4+three*(pool[c])^3+two*(pool[d])^2+one*(pool[e])^1+zero)
                            if child.degree(v)<=5:
                                break
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                            child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                            child = polynomial_modulo(child,2,v)
                            if child.degree(v) == 6 and child in field_of_degree_q:
                                field_of_degree_q.remove(child)
                            if len(field_of_degree_q)==0:
                                return []
                        
    return field_of_degree_q
            

    

In [18]:
def find_four(irre,v):
    field = GF(2,4,v);
    pool = GF(2,4,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^4+j)

    for mother in field:
        
        coe = mother.coefficients()

        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        
        for b in range(16):
            for c in range(16):
                for d in range (16):
                    child = expand(three*(pool[b])^3+two*(pool[c])^2+one*(pool[d])^1+zero)
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                    #child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                    child = polynomial_modulo(child,2,v)
                    if child.degree(v) == 4 and child in field_of_degree_q:
                        field_of_degree_q.remove(child)
    return field_of_degree_q
            


In [19]:
v = var('v')
for irre in degree_of_q_polynomials(2,5,v):
    poly = find_four(irre,v)
    print(poly)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [99]:
find_four(v^5 + v^4 + v^2 + v + 1,v)

[0, 1, v, v + 1, v^2, v^2 + 1, v^2 + v, v^2 + v + 1, v^3, v^3 + 1, v^3 + v, v^3 + v + 1, v^3 + v^2, v^3 + v^2 + 1, v^3 + v^2 + v, v^3 + v^2 + v + 1]
[v^4, v^4 + 1, v^4 + v, v^4 + v + 1, v^4 + v^2, v^4 + v^2 + 1, v^4 + v^2 + v, v^4 + v^2 + v + 1, v^4 + v^3, v^4 + v^3 + 1, v^4 + v^3 + v, v^4 + v^3 + v + 1, v^4 + v^3 + v^2, v^4 + v^3 + v^2 + 1, v^4 + v^3 + v^2 + v, v^4 + v^3 + v^2 + v + 1]
0
1
v
v + 1
v^2
v^2 + 1
v^2 + v
v^2 + v + 1
v^3
v^3 + 1
v^3 + v
v^3 + v + 1
v^3 + v^2
v^3 + v^2 + 1
v^3 + v^2 + v
v^3 + v^2 + v + 1


[v^4 + v,
 v^4 + v + 1,
 v^4 + v^2 + v,
 v^4 + v^3 + v,
 v^4 + v^3 + v + 1,
 v^4 + v^3 + v^2 + v + 1]

In [136]:
def find_three(irre,v):
    field = GF(2,3,v);
    pool = GF(2,3,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^3+j)

    
    for mother in field:
        
        coe = mother.coefficients()


        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        
        for b in range(8):
            for c in range(8):
                for d in range (8):
                    child = expand(two*(pool[c])^2+one*(pool[d])^1+zero)
                    
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                    child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                    child = polynomial_modulo(child,2,v)
                    if child.degree(v) == 3 and child in field_of_degree_q:
                        field_of_degree_q.remove(child)
    return field_of_degree_q
            

In [16]:
def find_seven(irre,v):
    print(irre)
    field = GF(2,7,v);
    pool = GF(2,7,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^7+j)

    for mother in reversed(field):
        

        coe = mother.coefficients()
        if (len(coe)>=7):
            six = coe[6][0] 
        else:
            six = 0
        if (len(coe)>=6):
            five = coe[5][0] 
        else:
            five = 0
        if (len(coe)>=5):
            four = coe[4][0] 
        else:
            four = 0
        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        
        for a in range(128):
            for b in range(128):
                for c in range(128):
                    for d in range (128):
                        for e in range(128):
                            for f in range(128):
                                child = expand(six*(pool[a])^6+five*(pool[b])^5+four*(pool[c])^4+three*(pool[d])^3+two*(pool[e])^2+one*(pool[f])^1+zero)
                                if child.degree(v)<=6:
                                    break
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                                #child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                                child = polynomial_modulo(child,2,v)
                                if child.degree(v) == 7 and child in field_of_degree_q:
                                    field_of_degree_q.remove(child)
                                if len(field_of_degree_q)==0:
                                    return []
                        
    return field_of_degree_q
            

    

In [17]:
v = var('v')
for irre in degree_of_q_polynomials(2,8,v):
    poly = find_seven(irre,v)
    print(poly)

v^8


KeyboardInterrupt: 

In [24]:
def find_eight(irre,v):
    print(irre)
    field = GF(2,8,v);
    pool = GF(2,8,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^8+j)

    for mother in reversed(field):
        

        coe = mother.coefficients()
        if (len(coe)>=8):
            seven = coe[7][0] 
        else:
            seven = 0
        if (len(coe)>=7):
            six = coe[6][0] 
        else:
            six = 0
        if (len(coe)>=6):
            five = coe[5][0] 
        else:
            five = 0
        if (len(coe)>=5):
            four = coe[4][0] 
        else:
            four = 0
        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        
        for a in range(256):
            for b in range(256):
                for c in range(256):
                    for d in range (256):
                        for e in range(256):
                            for f in range(256):
                                for g in range (256):
                                    child = expand(seven*(pool[a])^7+six*(pool[b])^6+five*(pool[c])^5+four*(pool[d])^4+three*(pool[e])^3+two*(pool[f])^2+one*(pool[g])^1+zero)
                                    if child.degree(v)<=7:
                                        break
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                                    child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                                    child = polynomial_modulo(child,2,v)
                                    if child.degree(v) == 8 and child in field_of_degree_q:
                                        field_of_degree_q.remove(child)
                                    if len(field_of_degree_q)==0:
                                        return []
                        
    return field_of_degree_q
            


In [25]:
v = var('v')
for irre in degree_of_q_polynomials(2,9,v):
    poly = find_eight(irre,v)
    print(poly)

v^9
[]
v^9 + 1
[]
v^9 + v
[]
v^9 + v + 1
[]
v^9 + v^2
[]
v^9 + v^2 + 1
[]
v^9 + v^2 + v
[]
v^9 + v^2 + v + 1
[]
v^9 + v^3
[]
v^9 + v^3 + 1
[]
v^9 + v^3 + v
[]
v^9 + v^3 + v + 1
[]
v^9 + v^3 + v^2
[]
v^9 + v^3 + v^2 + 1
[]
v^9 + v^3 + v^2 + v
[]
v^9 + v^3 + v^2 + v + 1
[]
v^9 + v^4
[]
v^9 + v^4 + 1
[]
v^9 + v^4 + v
[]
v^9 + v^4 + v + 1
[]
v^9 + v^4 + v^2
[]
v^9 + v^4 + v^2 + 1
[]
v^9 + v^4 + v^2 + v
[]
v^9 + v^4 + v^2 + v + 1
[]
v^9 + v^4 + v^3
[]
v^9 + v^4 + v^3 + 1
[]
v^9 + v^4 + v^3 + v
[]
v^9 + v^4 + v^3 + v + 1
[]
v^9 + v^4 + v^3 + v^2
[]
v^9 + v^4 + v^3 + v^2 + 1
[]
v^9 + v^4 + v^3 + v^2 + v
[]
v^9 + v^4 + v^3 + v^2 + v + 1
[]
v^9 + v^5
[]
v^9 + v^5 + 1
[]
v^9 + v^5 + v
[]
v^9 + v^5 + v + 1
[]
v^9 + v^5 + v^2
[]
v^9 + v^5 + v^2 + 1
[]
v^9 + v^5 + v^2 + v
[]
v^9 + v^5 + v^2 + v + 1
[]
v^9 + v^5 + v^3
[]
v^9 + v^5 + v^3 + 1
[]
v^9 + v^5 + v^3 + v
[]
v^9 + v^5 + v^3 + v + 1
[]
v^9 + v^5 + v^3 + v^2
[]
v^9 + v^5 + v^3 + v^2 + 1
[]
v^9 + v^5 + v^3 + v^2 + v
[]
v^9 + v^5 + v^3 + v^2 + v

[]
v^9 + v^8 + v^4 + v^3 + v^2 + v + 1
[]
v^9 + v^8 + v^5
[]
v^9 + v^8 + v^5 + 1
[]
v^9 + v^8 + v^5 + v
[]
v^9 + v^8 + v^5 + v + 1
[]
v^9 + v^8 + v^5 + v^2
[]
v^9 + v^8 + v^5 + v^2 + 1
[]
v^9 + v^8 + v^5 + v^2 + v
[]
v^9 + v^8 + v^5 + v^2 + v + 1
[]
v^9 + v^8 + v^5 + v^3
[]
v^9 + v^8 + v^5 + v^3 + 1
[]
v^9 + v^8 + v^5 + v^3 + v
[]
v^9 + v^8 + v^5 + v^3 + v + 1
[]
v^9 + v^8 + v^5 + v^3 + v^2
[]
v^9 + v^8 + v^5 + v^3 + v^2 + 1
[]
v^9 + v^8 + v^5 + v^3 + v^2 + v
[]
v^9 + v^8 + v^5 + v^3 + v^2 + v + 1
[]
v^9 + v^8 + v^5 + v^4
[]
v^9 + v^8 + v^5 + v^4 + 1
[]
v^9 + v^8 + v^5 + v^4 + v
[]
v^9 + v^8 + v^5 + v^4 + v + 1
[]
v^9 + v^8 + v^5 + v^4 + v^2
[]
v^9 + v^8 + v^5 + v^4 + v^2 + 1
[]
v^9 + v^8 + v^5 + v^4 + v^2 + v
[]
v^9 + v^8 + v^5 + v^4 + v^2 + v + 1
[]
v^9 + v^8 + v^5 + v^4 + v^3
[]
v^9 + v^8 + v^5 + v^4 + v^3 + 1
[]
v^9 + v^8 + v^5 + v^4 + v^3 + v
[]
v^9 + v^8 + v^5 + v^4 + v^3 + v + 1
[]
v^9 + v^8 + v^5 + v^4 + v^3 + v^2
[]
v^9 + v^8 + v^5 + v^4 + v^3 + v^2 + 1
[]
v^9 + v^8 + v^5 + v^

In [30]:
def find_nine(irre,v):
    print(irre)
    field = GF(2,9,v);
    pool = GF(2,9,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^9+j)

    for mother in reversed(field):
        

        coe = mother.coefficients()
        if (len(coe)>=9):
            eight = coe[8][0] 
        else:
            eight = 0
        if (len(coe)>=8):
            seven = coe[7][0] 
        else:
            seven = 0
        if (len(coe)>=7):
            six = coe[6][0] 
        else:
            six = 0
        if (len(coe)>=6):
            five = coe[5][0] 
        else:
            five = 0
        if (len(coe)>=5):
            four = coe[4][0] 
        else:
            four = 0
        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        
        for a in range(len(pool)):
            for b in range(len(pool)):
                for c in range(len(pool)):
                    for d in range (len(pool)):
                        for e in range(len(pool)):
                            for f in range(len(pool)):
                                for g in range (len(pool)):
                                    for h in range(len(pool)):
                                        child = expand(eight*(pool[a])^8+seven*(pool[b])^7+six*(pool[c])^6+five*(pool[d])^5+four*(pool[e])^4+three*(pool[f])^3+two*(pool[g])^2+one*(pool[h])^1+zero)
                                        if child.degree(v)<=8:
                                            break
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                                        child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                                        child = polynomial_modulo(child,2,v)
                                        if child.degree(v) == 9 and child in field_of_degree_q:
                                            field_of_degree_q.remove(child)
                                            print(len(field_of_degree_q))
                                        if len(field_of_degree_q)==0:
                                            return []
                        
    return field_of_degree_q
            


In [ ]:
v = var('v')
for irre in degree_of_q_polynomials(2,10,v):
    poly = find_nine(irre,v)
    print(poly)

v^10


In [11]:
def find_ten(irre,v):
    print(irre)
    field = GF(2,10,v);
    pool = GF(2,10,v);
    field_of_degree_q = []
    for i in range(1,2):
        for j in field:
            field_of_degree_q.append(i*v^10+j)

    for mother in reversed(field):
        

        coe = mother.coefficients()
        if (len(coe)>=10):
            nine = coe[9][0] 
        else:
            nine = 0
        if (len(coe)>=9):
            eight = coe[8][0] 
        else:
            eight = 0
        if (len(coe)>=8):
            seven = coe[7][0] 
        else:
            seven = 0
        if (len(coe)>=7):
            six = coe[6][0] 
        else:
            six = 0
        if (len(coe)>=6):
            five = coe[5][0] 
        else:
            five = 0
        if (len(coe)>=5):
            four = coe[4][0] 
        else:
            four = 0
        if (len(coe)>=4):
            three = coe[3][0]
        else:
            three = 0
        if (len(coe)>=3):
            two =coe[2][0] 
        else:
            two = 0
        if (len(coe)>=2):
            one =coe[1][0]
        else:
            one = 0
        zero = coe[0][0]
        
        
        for a in range(len(pool)):
            for b in range(len(pool)):
                for c in range(len(pool)):
                    for d in range (len(pool)):
                        for e in range(len(pool)):
                            for f in range(len(pool)):
                                for g in range (len(pool)):
                                    for h in range(len(pool)):
                                        for i in range(len(pool)):
                                            child = expand(nine*(pool[a])^9+eight*(pool[b])^8+seven*(pool[c])^7+six*(pool[d])^6+five*(pool[e])^5+four*(pool[f])^4+three*(pool[g])^3+two*(pool[h])^2+one*(pool[i])^1+zero)
                                            if child.degree(v)<=9:
                                                break
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
                                            child = polynomial_division(child,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
                                            child = polynomial_modulo(child,2,v)
                                            if child.degree(v) == 10 and child in field_of_degree_q:
                                                field_of_degree_q.remove(child)
                                                
                                            if len(field_of_degree_q)==0:
                                                return []
                        
    return field_of_degree_q
            

In [12]:
v = var('v')
for irre in degree_of_q_polynomials(2,11,v):
    poly = find_ten(irre,v)
    print(poly)

v^11
[]
v^11 + 1
[]
v^11 + v
[]
v^11 + v + 1
[]
v^11 + v^2
[]
v^11 + v^2 + 1
[]
v^11 + v^2 + v
[]
v^11 + v^2 + v + 1
[]
v^11 + v^3
[]
v^11 + v^3 + 1
[]
v^11 + v^3 + v
[]
v^11 + v^3 + v + 1
[]
v^11 + v^3 + v^2
[]
v^11 + v^3 + v^2 + 1
[]
v^11 + v^3 + v^2 + v
[]
v^11 + v^3 + v^2 + v + 1
[]
v^11 + v^4
[]
v^11 + v^4 + 1
[]
v^11 + v^4 + v
[]
v^11 + v^4 + v + 1
[]
v^11 + v^4 + v^2
[]
v^11 + v^4 + v^2 + 1
[]
v^11 + v^4 + v^2 + v
[]
v^11 + v^4 + v^2 + v + 1
[]
v^11 + v^4 + v^3
[]
v^11 + v^4 + v^3 + 1
[]
v^11 + v^4 + v^3 + v
[]
v^11 + v^4 + v^3 + v + 1
[]
v^11 + v^4 + v^3 + v^2
[]
v^11 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^4 + v^3 + v^2 + v
[]
v^11 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^5
[]
v^11 + v^5 + 1
[]
v^11 + v^5 + v
[]
v^11 + v^5 + v + 1
[]
v^11 + v^5 + v^2
[]
v^11 + v^5 + v^2 + 1
[]
v^11 + v^5 + v^2 + v
[]
v^11 + v^5 + v^2 + v + 1
[]
v^11 + v^5 + v^3
[]
v^11 + v^5 + v^3 + 1
[]
v^11 + v^5 + v^3 + v
[]
v^11 + v^5 + v^3 + v + 1
[]
v^11 + v^5 + v^3 + v^2
[]
v^11 + v^5 + v^3 + v^2 + 1
[]
v^11 + v

[]
v^11 + v^8 + v^4 + v^2 + v
[]
v^11 + v^8 + v^4 + v^2 + v + 1
[]
v^11 + v^8 + v^4 + v^3
[]
v^11 + v^8 + v^4 + v^3 + 1
[]
v^11 + v^8 + v^4 + v^3 + v
[]
v^11 + v^8 + v^4 + v^3 + v + 1
[]
v^11 + v^8 + v^4 + v^3 + v^2
[]
v^11 + v^8 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^8 + v^4 + v^3 + v^2 + v
[]
v^11 + v^8 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^8 + v^5
[]
v^11 + v^8 + v^5 + 1
[]
v^11 + v^8 + v^5 + v
[]
v^11 + v^8 + v^5 + v + 1
[]
v^11 + v^8 + v^5 + v^2
[]
v^11 + v^8 + v^5 + v^2 + 1
[]
v^11 + v^8 + v^5 + v^2 + v
[]
v^11 + v^8 + v^5 + v^2 + v + 1
[]
v^11 + v^8 + v^5 + v^3
[]
v^11 + v^8 + v^5 + v^3 + 1
[]
v^11 + v^8 + v^5 + v^3 + v
[]
v^11 + v^8 + v^5 + v^3 + v + 1
[]
v^11 + v^8 + v^5 + v^3 + v^2
[]
v^11 + v^8 + v^5 + v^3 + v^2 + 1
[]
v^11 + v^8 + v^5 + v^3 + v^2 + v
[]
v^11 + v^8 + v^5 + v^3 + v^2 + v + 1
[]
v^11 + v^8 + v^5 + v^4
[]
v^11 + v^8 + v^5 + v^4 + 1
[]
v^11 + v^8 + v^5 + v^4 + v
[]
v^11 + v^8 + v^5 + v^4 + v + 1
[]
v^11 + v^8 + v^5 + v^4 + v^2
[]
v^11 + v^8 + v^5 + v^4 + v^2 + 1
[]
v

[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^2 + v + 1
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + 1
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + v
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + v + 1
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + v^2
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + v^2 + v
[]
v^11 + v^8 + v^7 + v^6 + v^5 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^9
[]
v^11 + v^9 + 1
[]
v^11 + v^9 + v
[]
v^11 + v^9 + v + 1
[]
v^11 + v^9 + v^2
[]
v^11 + v^9 + v^2 + 1
[]
v^11 + v^9 + v^2 + v
[]
v^11 + v^9 + v^2 + v + 1
[]
v^11 + v^9 + v^3
[]
v^11 + v^9 + v^3 + 1
[]
v^11 + v^9 + v^3 + v
[]
v^11 + v^9 + v^3 + v + 1
[]
v^11 + v^9 + v^3 + v^2
[]
v^11 + v^9 + v^3 + v^2 + 1
[]
v^11 + v^9 + v^3 + v^2 + v
[]
v^11 + v^9 + v^3 + v^2 + v + 1
[]
v^11 + v^9 + v^4
[]
v^11 + v^9 + v^4 + 1
[]
v^11 + v^9 + v^4 + v
[]
v^11 + v^9 + v^4 + v + 1
[]
v^11 + v^9 + v^4 + v^2
[]
v^11 + v^9 + v^4 + v^2 + 1
[]
v

[]
v^11 + v^9 + v^7 + v^6 + v^5
[]
v^11 + v^9 + v^7 + v^6 + v^5 + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^2
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^2 + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^2 + v
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^2 + v + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + v
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + v + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + v^2
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + v^2 + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + v^2 + v
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^3 + v^2 + v + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4 + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4 + v
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4 + v + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4 + v^2
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4 + v^2 + 1
[]
v^11 + v^9 + v^7 + v^6 + v^5 + v^4 + v^2 + v
[]
v^11 + v^9 + v^7 

[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^3 + v^2 + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^3 + v^2 + v
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^3 + v^2 + v + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^2
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^2 + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^2 + v
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^2 + v + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + v
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + v + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + v^2
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + v^2 + v
[]
v^11 + v^9 + v^8 + v^7 + v^5 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^9 + v^8 + v^7 + v^6
[]
v^11 + v^9 + v^8 + v^7 + v^6 + 1
[]
v^11 + v^9 + 

[]
v^11 + v^10 + v^6 + v^5 + v^4 + v^3 + v^2 + v
[]
v^11 + v^10 + v^6 + v^5 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^7
[]
v^11 + v^10 + v^7 + 1
[]
v^11 + v^10 + v^7 + v
[]
v^11 + v^10 + v^7 + v + 1
[]
v^11 + v^10 + v^7 + v^2
[]
v^11 + v^10 + v^7 + v^2 + 1
[]
v^11 + v^10 + v^7 + v^2 + v
[]
v^11 + v^10 + v^7 + v^2 + v + 1
[]
v^11 + v^10 + v^7 + v^3
[]
v^11 + v^10 + v^7 + v^3 + 1
[]
v^11 + v^10 + v^7 + v^3 + v
[]
v^11 + v^10 + v^7 + v^3 + v + 1
[]
v^11 + v^10 + v^7 + v^3 + v^2
[]
v^11 + v^10 + v^7 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^7 + v^3 + v^2 + v
[]
v^11 + v^10 + v^7 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^7 + v^4
[]
v^11 + v^10 + v^7 + v^4 + 1
[]
v^11 + v^10 + v^7 + v^4 + v
[]
v^11 + v^10 + v^7 + v^4 + v + 1
[]
v^11 + v^10 + v^7 + v^4 + v^2
[]
v^11 + v^10 + v^7 + v^4 + v^2 + 1
[]
v^11 + v^10 + v^7 + v^4 + v^2 + v
[]
v^11 + v^10 + v^7 + v^4 + v^2 + v + 1
[]
v^11 + v^10 + v^7 + v^4 + v^3
[]
v^11 + v^10 + v^7 + v^4 + v^3 + 1
[]
v^11 + v^10 + v^7 + v^4 + v^3 + v
[]
v^11 + v^10 + v^7 + 

[]
v^11 + v^10 + v^8 + v^6 + v^4 + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^2
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^2 + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^2 + v
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^2 + v + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + v
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + v + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + v^2
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + v^2 + v
[]
v^11 + v^10 + v^8 + v^6 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^8 + v^6 + v^5
[]
v^11 + v^10 + v^8 + v^6 + v^5 + 1
[]
v^11 + v^10 + v^8 + v^6 + v^5 + v
[]
v^11 + v^10 + v^8 + v^6 + v^5 + v + 1
[]
v^11 + v^10 + v^8 + v^6 + v^5 + v^2
[]
v^11 + v^10 + v^8 + v^6 + v^5 + v^2 + 1
[]
v^11 + v^10 + v^8 + v^6 + v^5 + v^2 + v
[]
v^11 + v^10 + v^8 + v^6 + v^5 + v^2 + v + 1
[]
v^11 + v^10 + v^8 + v^

[]
v^11 + v^10 + v^9 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^3
[]
v^11 + v^10 + v^9 + v^3 + 1
[]
v^11 + v^10 + v^9 + v^3 + v
[]
v^11 + v^10 + v^9 + v^3 + v + 1
[]
v^11 + v^10 + v^9 + v^3 + v^2
[]
v^11 + v^10 + v^9 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^3 + v^2 + v
[]
v^11 + v^10 + v^9 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^4
[]
v^11 + v^10 + v^9 + v^4 + 1
[]
v^11 + v^10 + v^9 + v^4 + v
[]
v^11 + v^10 + v^9 + v^4 + v + 1
[]
v^11 + v^10 + v^9 + v^4 + v^2
[]
v^11 + v^10 + v^9 + v^4 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^4 + v^2 + v
[]
v^11 + v^10 + v^9 + v^4 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^4 + v^3
[]
v^11 + v^10 + v^9 + v^4 + v^3 + 1
[]
v^11 + v^10 + v^9 + v^4 + v^3 + v
[]
v^11 + v^10 + v^9 + v^4 + v^3 + v + 1
[]
v^11 + v^10 + v^9 + v^4 + v^3 + v^2
[]
v^11 + v^10 + v^9 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^4 + v^3 + v^2 + v
[]
v^11 + v^10 + v^9 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^5
[]
v^11 + v^10 + v^9 + v^5 + 1
[]
v^11 + v^10 + v^9 + v^5 + v
[]
v^11 + v^

[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^3 + v^2 + v
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^2
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^2 + v
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + v
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + v + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + v^2
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + v^2 + v
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^7 + v^6 + v^5
[]
v^11 + v^10 + v^9 + v^7

[]
v^11 + v^10 + v^9 + v^8 + v^6 + v^5 + v^4 + v^3 + v^2
[]
v^11 + v^10 + v^9 + v^8 + v^6 + v^5 + v^4 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^8 + v^6 + v^5 + v^4 + v^3 + v^2 + v
[]
v^11 + v^10 + v^9 + v^8 + v^6 + v^5 + v^4 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7
[]
v^11 + v^10 + v^9 + v^8 + v^7 + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^2
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^2 + v
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + v
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + v + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + v^2
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + v^2 + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + v^2 + v
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^3 + v^2 + v + 1
[]
v^11 + v^10 + v^9 + v^8 + v^7 + v^4
[]
v^11 + v^10 + v^9 + v^8 + 

In [ ]:
var('x,y,z')
f=x^3+y^2+z

p1=x+1
p2=x^2+x+1
p3=x^2+1

f.sub()

for each poly in:
    1000000000
    1000000001
    1000000010



1010101010

kv=v^10+v^8+v^6
fx = x^5+x^3+x
gx([x^2, ]
   mod dx
   
   
   